<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-08 13:23:08
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.33 C
-------------------
Today PnL: -38.33 K (-0.29%)
Current PnL: -21.77 L (-14.81%)
CY Booked + Current PnL: -13.50 L (-9.18%)
-------------------
Total profit:  2.09 L
Total loss:  -23.86 L
-------------------
Total Booked + Current PnL: 13.48 L (11.23%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.21%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 80.03 L (60.12%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,26.82,H-SC,13.74,162620.0,36420.0,9757.0,5.64,28.86,6.00,36.59,156.0,3.73,1.26,44.99,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.64,107936.0,3624.0,13028.0,-1.06,3.47,12.07,15.96,10.0,0.28,0.83,20.96,X5K,ATH,METALS
71,SIS,477.00,477.00,2275.13,H-SC,7.13,63903.0,-17187.0,17190.0,1.18,-21.19,26.90,0.00,157.0,-1.00,0.49,29.51,OX40N,NTT,MISC
82,TTKPRESTIG,769.29,770.00,-9.13,H-SC,11.15,83074.0,-17703.0,17794.0,0.02,-17.57,21.42,0.09,145.0,-0.99,0.64,9.14,OX40N,NTT,DURABLES
67,SBIN,760.30,863.00,-13.42,M-LC,3.86,210582.0,9102.0,18110.0,-1.30,4.52,8.60,13.51,62.0,0.50,1.63,17.68,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,PGHH,13388.0,18062.58,-20.87,H-MC,6.64,199800.0,-1020.0,71129.0,0.78,-0.51,35.6,34.92,84.0,-0.01,1.54,4.14,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.50,H-LC,1.94,204130.0,-370.0,41785.0,-0.07,-0.18,20.47,20.25,9.0,-0.01,1.58,8.54,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-14.54,H-LC,2.46,169864.0,-31522.0,119143.0,-0.34,-15.65,70.14,43.51,19.0,-0.26,1.31,18.24,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-5.95,H-LC,3.52,181080.0,-19230.0,79204.0,-0.15,-9.60,43.74,29.94,24.0,-0.24,1.40,25.10,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-59.03,H-LC,3.87,129882.0,-135544.0,192459.0,-0.64,-51.07,148.18,21.44,26.0,-0.70,1.00,4.37,XY25,NTT,FMCG
10,BAJAJHFL,122.26,152.0,-10.62,H-LC,4.10,185369.0,-15137.0,63915.0,-0.24,-7.55,34.48,24.33,30.0,-0.24,1.43,6.89,X40N,BTT,FINANCE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,26.82,H-SC,13.74,162620.0,36420.0,9757.0,5.64,28.86,6.00,36.59,156.0,3.73,1.26,44.99,XY24,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.0,-51.71,L-SC,22.06,70906.0,-42643.0,82698.0,4.20,-37.55,116.63,35.27,267.0,-0.52,0.55,73.54,XR,NTT,HOTELS
83,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.69,119556.0,-22923.0,23541.0,2.41,-16.09,19.69,0.44,163.0,-0.97,0.92,41.11,OX40N,NTT,BANKS
86,VAIBHAVGBL,350.21,521.0,44.90,H-SC,7.84,96565.0,-55426.0,129552.0,1.98,-36.47,134.16,48.77,127.0,-0.43,0.75,11.62,XR,NTT,JEWELLERY
8,AWL,325.39,485.0,-69.63,H-MC,5.24,210705.0,-57742.0,189424.0,1.87,-21.51,89.90,49.05,82.0,-0.30,1.63,8.34,XY24,NTT,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LAOPALA,369.40,464.00,115.96,H-SC,3.48,68700.0,-32146.0,57969.0,-4.77,-31.88,84.38,25.61,141.0,-0.55,0.53,27.37,AR,NTT,CERAMICS
17,BSOFT,628.85,836.99,-22.90,H-SC,13.76,64636.0,-45413.0,81836.0,-3.30,-41.27,126.61,33.10,129.0,-0.55,0.50,7.40,XR,ATH,IT
80,TITAGARH,1097.23,1548.00,-11.74,H-SC,6.08,144699.0,-47316.0,126206.0,-3.17,-24.64,87.22,41.08,158.0,-0.37,1.12,22.47,XY24,NTT,ENGINEERING
27,DIXON,14367.00,18940.15,-29.66,H-MC,6.57,129480.0,14544.0,22037.0,-2.87,12.65,17.02,31.83,85.0,0.66,1.00,41.36,X40N,ATH,IT
90,WHIRLPOOL,1219.98,2270.00,-43.35,M-SC,4.89,95318.0,3820.0,74929.0,-1.84,4.17,78.61,86.07,211.0,0.05,0.74,37.89,XR,NTT,DURABLES


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,124.89,H-SC,3.91,147560.0,-26999.0,27033.0,-1.66,-15.47,18.32,0.02,135.0,-1.00,1.14,27.96,OX40N,NTT,PAINTS
83,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.69,119556.0,-22923.0,23541.0,2.41,-16.09,19.69,0.44,163.0,-0.97,0.92,41.11,OX40N,NTT,BANKS
82,TTKPRESTIG,769.29,770.0,-9.13,H-SC,11.15,83074.0,-17703.0,17794.0,0.02,-17.57,21.42,0.09,145.0,-0.99,0.64,9.14,OX40N,NTT,DURABLES
77,TATAELXSI,7332.28,7332.0,-22.35,H-MC,2.13,81452.0,-21200.0,21194.0,-0.80,-20.65,26.02,-0.00,101.0,-1.00,0.63,23.82,OX40N,NTT,IT
71,SIS,477.00,477.0,2275.13,H-SC,7.13,63903.0,-17187.0,17190.0,1.18,-21.19,26.90,0.00,157.0,-1.00,0.49,29.51,OX40N,NTT,MISC


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.06,H-LC,15.37,237354.0,-58580.0,152500.0,-0.13,-19.80,64.25,31.73,1.0,-0.38,1.83,1.33,X40,BTT,IT
88,VBL,492.64,671.64,-11.37,H-LC,11.01,269134.0,4094.0,92205.0,-0.49,1.54,34.26,36.33,3.0,0.04,2.08,15.05,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-20.32,H-LC,9.50,255290.0,-6311.0,151923.0,-0.74,-2.41,59.51,55.67,4.0,-0.04,1.97,3.55,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.34,H-LC,9.41,198705.0,-17814.0,110718.0,-0.27,-8.23,55.72,42.91,5.0,-0.16,1.53,4.06,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,1.41,H-LC,9.89,180096.0,-10550.0,57054.0,0.05,-5.53,31.68,24.40,8.0,-0.18,1.39,9.52,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,26.82,H-SC,13.74,162620.0,36420.0,9757.0,5.64,28.86,6.0,36.59,156.0,3.73,1.26,44.99,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,26.82,H-SC,13.74,162620.0,36420.0,9757.0,5.64,28.86,6.00,36.59,156.0,3.73,1.26,44.99,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.64,25.50,H-SC,13.22,128394.0,3097.0,55980.0,-0.63,2.47,43.60,47.15,144.0,0.06,0.99,60.63,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,0.95,H-MC,6.09,162470.0,11557.0,45752.0,-0.40,7.66,28.16,37.97,90.0,0.25,1.25,34.90,AR,ATH,AUTO
15,BLUESTARCO,1646.70,2326.38,-3.89,H-SC,7.95,180400.0,15730.0,52244.0,-1.29,9.55,28.96,41.28,119.0,0.30,1.39,18.30,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-5.69,H-SC,2.38,105916.0,3912.0,40015.0,-0.66,3.84,37.78,43.06,120.0,0.10,0.82,21.61,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,0.95,H-MC,6.09,162470.0,11557.0,45752.0,-0.40,7.66,28.16,37.97,90.0,0.25,1.25,34.90,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-47.89,H-SC,13.34,135595.0,12259.0,121507.0,-0.07,9.94,89.61,108.45,118.0,0.10,1.05,34.51,AR,ATH,MISC
50,KPIGREEN,497.21,731.64,25.50,H-SC,13.22,128394.0,3097.0,55980.0,-0.63,2.47,43.60,47.15,144.0,0.06,0.99,60.63,MH,ATH,POWER
90,WHIRLPOOL,1219.98,2270.00,-43.35,M-SC,4.89,95318.0,3820.0,74929.0,-1.84,4.17,78.61,86.07,211.0,0.05,0.74,37.89,XR,NTT,DURABLES
91,WIPRO,243.46,420.00,-12.85,M-LC,6.48,149408.0,-1537.0,110995.0,-0.55,-1.02,74.29,72.51,55.0,-0.01,1.15,4.94,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2275.13,H-SC,7.13,63903.0,-17187.0,17190.0,1.18,-21.19,26.90,0.00,157.0,-1.00,0.49,29.51,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,22.06,70906.0,-42643.0,82698.0,4.20,-37.55,116.63,35.27,267.0,-0.52,0.55,73.54,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.83,H-SC,17.92,89548.0,-11415.0,105944.0,-1.22,-11.31,118.31,93.63,149.0,-0.11,0.69,33.38,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-18.70,H-SC,16.08,92325.0,-5655.0,27522.0,1.58,-5.77,29.81,22.32,151.0,-0.21,0.71,33.90,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.64,107936.0,3624.0,13028.0,-1.06,3.47,12.07,15.96,10.0,0.28,0.83,20.96,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,26.82,H-SC,13.74,162620.0,36420.0,9757.0,5.64,28.86,6.00,36.59,156.0,3.73,1.26,44.99,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-23.26,H-SC,8.08,218436.0,25416.0,22630.0,-0.03,13.17,10.36,24.89,128.0,1.12,1.69,39.59,X40N,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,946.25,M-SC,8.15,142384.0,-38759.0,84234.0,-0.13,-21.40,59.16,25.10,233.0,-0.46,1.10,14.11,XY24,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,22.06,70906.0,-42643.0,82698.0,4.20,-37.55,116.63,35.27,267.0,-0.52,0.55,73.54,XR,NTT,HOTELS
35,HEROMOTOCO,4311.81,5949.07,0.95,H-MC,6.09,162470.0,11557.0,45752.0,-0.40,7.66,28.16,37.97,90.0,0.25,1.25,34.90,AR,ATH,AUTO


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.42
1,25,41.28
2,50,71.38


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.08
LC    36.55
MC    22.39
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.24
X40N     15.41
X40      13.49
XY25     10.81
XR        9.06
AR        7.76
OX40N     7.71
X5K       2.25
MH        1.76
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.32
H-SC    27.65
H-MC    19.41
M-SC    11.96
M-LC     7.14
M-MC     2.63
L-SC     1.47
L-LC     1.09
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.40,-10.60,46.16
IT,12.07,-19.32,77.28
FINANCE,8.40,-19.86,63.35
BANKS,7.62,-15.55,62.69
PAINTS,5.79,-16.28,39.20
HEALTHCARE,5.26,3.44,20.94
MISC,5.20,-21.39,92.34
ELECTRICAL,4.77,-13.55,62.73
AUTO,4.44,-17.24,68.24


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2850826.0,23
XR,1118868.0,13
AR,1059280.0,9
X40,823835.0,10
XY25,649418.0,8
X40N,613477.0,12
OX40N,376786.0,10
SR,196698.0,2
X5K,146523.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2741357.0,30
H-LC,1486505.0,20
M-SC,1420932.0,15
H-MC,1189090.0,15
M-LC,453415.0,5
M-MC,336873.0,2
L-SC,271106.0,3
L-MC,57390.0,1
L-LC,45892.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      945534.0      7
M-SC       XY24      862004.0      7
H-SC       AR        620428.0      4
H-LC       X40       548355.0      6
H-SC       XR        523354.0      6
H-MC       XY24      470526.0      4
M-MC       XY24      336873.0      2
H-LC       X40N      316553.0      6
           XY25      251490.0      3
           AR        232801.0      2
H-MC       XY25      210898.0      2
H-SC       X40N      208622.0      4
H-MC       X40       199512.0      3
H-SC       SR        196698.0      2
M-LC       XY24      190815.0      2
H-SC       OX40N     190741.0      6
L-SC       XR        181385.0      2
M-SC       AR        160299.0      2
M-LC       X5K       133495.0      1
M-SC       XR        124503.0      2
           XY25      123028.0      1
H-MC       XR        121241.0      1
M-LC       XR        110995.0      1
L-SC       OX40N      89721.0      1
H-MC       X40N       88302.0      2
H-LC       X200       79204.0      1
M-SC       X40        75968.0      1
           OX40N      75130.0      2
L-MC       XR         57390.0      1
H-SC       MH         55980.0      1
L-LC       XY25       45892.0      1
H-MC       AR         45752.0      1
H-LC       XY24       45074.0      1
H-MC       MH         31665.0      1
           OX40N      21194.0      1
M-LC       XY25       18110.0      1
H-LC       X5K        13028.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
